In [11]:
#import statements
import numpy as np
from astropy.io import ascii
from astropy.table import Table, Column
from astropy import units as u
import h5py

#read in results from MCMC fitting
results=ascii.read('/users/adignan/results_mcmc_filt.csv',format='csv')

#path to MCMC input files
path='/users/adignan/mcmcfiles/'

#initialize empty lists
thermalfrac=[]
thermalfrac_unc=[]

#define equation for calculating free-free emission
def I_ff(nu, T, Aff):
    def gff(nu, T):
        T4 = T/(1.e4*u.K)
        nu9 = nu/(1.*u.GHz)
        gnu = np.log(np.exp(5.960 - (np.sqrt(3.)/np.pi)*np.log(nu9*(T4**(-3./2.)))) + np.exp(1.))
        return gnu   
    
    return Aff*gff(nu,T)/gff(30.*u.GHz,T)

#for loop to calculate thermal fractions and uncertainties
for row in results:
    if row['sourceid']=='NGC0337a':
    
        file=row['sourceid']

        #read in MCMC file
        (nu, fnu, sfnu) = np.genfromtxt(path+file, unpack=True)
        
        #calculate thermal fraction based on best-fit result
        print(row['A_ff median'])
        tfrac= I_ff(33*u.GHz,1.e4*u.K,row['A_ff median'])/fnu[2]
        thermalfrac.append(tfrac)

        #calculate uncertainty on thermal fraction

        #load in samples file
        hf = h5py.File('/users/adignan/hdf5/'+str(file)+'-samples_eric.h5', 'r')
        n1 = hf.get('dataset')
        n1=np.array(n1)

        #calculate a few different stats for the samples
        unc=np.std(n1[:,3])
        
        print(unc)

        #calculate 
        tfrac_unc=np.sqrt( (sfnu[2]/fnu[2])**2 + (unc/tfrac)**2 ) * tfrac
        thermalfrac_unc.append(tfrac_unc)

        #print statements for looking at results without saving it to a csv file
        print(r'tfrac: '+"%.3f" % tfrac + ' ± ' +"%.3f" % tfrac_unc)
        
        print(r'tfrac: '+ str(tfrac) + ' ± ' + str(tfrac_unc))
        
        hf.close

#make astropy table of results
# idscol=Column(results['sourceid'],name='source ID')
# fraccol=Column(thermalfrac,name='thermal fraction')
# unccol=Column(thermalfrac_unc,name='uncertainty')

# tbl1=Table([idscol,fraccol,unccol])
#write results to csv file
# tbl1.write('/users/adignan/thermalfractions_mcmc.csv',format='csv',overwrite=True)

0.2860688550537509
0.2099678728464002
tfrac: 0.467 ± 0.226
tfrac: 0.4672513457488573 ± 0.22623119251004403


In [8]:
import h5py
import numpy as np
from astropy import units as u

def I_ff(nu, T, Aff):
    def gff(nu, T):
        T4 = T/(1.e4*u.K)
        nu9 = nu/(1.*u.GHz)
        gnu = np.log(np.exp(5.960 - (np.sqrt(3.)/np.pi)*np.log(nu9*(T4**(-3./2.)))) + np.exp(1.))
        return gnu   
    
    return Aff*gff(nu,T)/gff(30.*u.GHz,T)

(nu, fnu, sfnu) = np.genfromtxt('/users/adignan/mcmcfiles/NGC0337a', unpack=True)

hf = h5py.File('/users/adignan/hdf5/NGC0337a-samples_eric.h5', 'r')
hf.keys()
samp = hf.get('dataset')
samp = np.array(samp)
samp.shape
hf.close

fnu33_ff_allfits = []

for i in samp[:,3]:
    Aff_fit = i
    tmp = I_ff(33*u.GHz, 1.e4*u.K, Aff_fit)
    fnu33_ff_allfits.append(tmp)
#     fnu33_ff_allfits = I_ff(33*u.GHz, 1.e4*u.K, Aff_fit)

Aff_mcmc=0.2860688550537509

#report: median; error on median, dispersion
# <x>, (pi/2)*mad(x)/sqrt(N), mad(x)
from astropy.stats import median_absolute_deviation as mad
fnu33 = fnu[2]
efnu33 = sfnu[2]
#fnu33_fit = np.interp(33*u.GHz, nu_out, snu_model)*u.mJy #f33 from best fit
fnu33_ff_fit = I_ff(33*u.GHz, 1.e4*u.K, Aff_mcmc) #f33^T from best fit
fnu33_ff_fit2 = np.median(fnu33_ff_allfits) #median of f33^T from all fits
efnu33_ff_fit_med = (np.pi/2)*np.std(fnu33_ff_allfits)/np.sqrt(len(fnu33_ff_allfits)) #error on median
efnu33_ff_fit = mad(fnu33_ff_allfits)/0.6745 #Scaled MAD f33^T from all fits
efnu33_ff_fit2 = np.std(fnu33_ff_allfits) #rms f33^T from all fits#print(fnu33_ff_fit, fnu33_ff_fit2)
#print(efnu33_ff_fit_med)
# print(efnu33_ff_fit, efnu33_ff_fit2)
tf33 = fnu33_ff_fit/fnu33 #33 GHz ff fracion
etf33 = np.sqrt((efnu33_ff_fit2/fnu33_ff_fit)**2 + (efnu33/fnu33)**2)*tf33
print(tf33.value, etf33.value)

0.4672513457488573 0.3531428093872606
